In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

In [ ]:
train_df = pd.read_csv("data/sign_mnist_train.csv")
test_df = pd.read_csv("data/sign_mnist_test.csv")

In [ ]:
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

x_train = train_df.values
x_test = test_df.values

x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [1]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)
history = model.fit(datagen.flow(x_train,y_train, batch_size = 128) ,epochs = 20 , validation_data = (x_test, y_test) , callbacks = [learning_rate_reduction])

model.save('smnist.h5')

2023-05-01 13:47:34.674120: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 75)       300       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 75)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 50)       2

2023-05-01 13:47:39.153634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
215/215 [==============================] - 20s 91ms/step - loss: 1.0879 - accuracy: 0.6617 - val_loss: 3.8524 - val_accuracy: 0.0958 - lr: 0.0010
Epoch 2/20
215/215 [==============================] - 20s 94ms/step - loss: 0.2075 - accuracy: 0.9328 - val_loss: 1.3424 - val_accuracy: 0.5672 - lr: 0.0010
Epoch 3/20
215/215 [==============================] - 20s 94ms/step - loss: 0.1038 - accuracy: 0.9666 - val_loss: 0.1548 - val_accuracy: 0.9525 - lr: 0.0010
Epoch 4/20
215/215 [==============================] - 21s 96ms/step - loss: 0.0612 - accuracy: 0.9807 - val_loss: 0.0193 - val_accuracy: 0.9947 - lr: 0.0010
Epoch 5/20
215/215 [==============================] - 23s 107ms/step - loss: 0.0462 - accuracy: 0.9859 - val_loss: 0.1771 - val_accuracy: 0.9398 - lr: 0.0010
Epoch 6/20
215/215 [==============================] - ETA: 0s - loss: 0.0372 - accuracy: 0.9877
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
215/215 [==============================] -

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 75)       300       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 75)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 50)       2

In [37]:
import random
import numpy as np
from keras.models import load_model


model = load_model('smnist.h5')


amount_of_samples = 500

# Pick 10 random images for test data set
random.seed(3) # to make this deterministic
sample_indexes = random.sample(range(len(x_test)), amount_of_samples)
sample_images = [x_test[i] for i in sample_indexes]
sample_labels = [y_test[i] for i in sample_indexes]

ground_truth = np.argmax(sample_labels, axis=1)

X_sample = np.array(sample_images)
prediction = model.predict(X_sample)
predicted_categories = np.argmax(prediction, axis=1)
predicted_categories

16/16 [==============================] - 0s 9ms/step


array([14, 12,  1, 19, 15, 22, 20, 22, 13, 17,  3, 13,  1,  4,  5, 11, 21,
       14,  4,  0, 14, 18,  6,  6, 12, 10, 23,  4,  2, 12, 17, 23, 23, 20,
       12, 11,  7, 22, 17,  3, 17,  6,  1, 22,  4, 23,  4,  6, 20, 14, 17,
       21,  5, 11,  2,  6, 15,  9,  1,  4, 20,  4, 23, 11,  1,  1,  8,  5,
        6, 11,  7,  7,  7,  3,  5, 20, 20,  7,  9, 18,  6, 21,  6,  4, 20,
       16, 15, 20,  5, 11,  6, 13,  7, 10,  4,  5, 18,  7, 19, 20,  5,  8,
       23,  2, 14, 21,  6, 15, 20, 20,  0,  8,  9,  4,  4, 23, 14,  4,  5,
       14,  1, 18, 17, 23, 13,  4, 20,  3,  9, 11,  7,  1,  9, 10,  9,  4,
       14,  3,  3,  0, 13,  2, 11, 21,  1,  4, 14, 22, 18,  7, 16, 12, 23,
        6,  6, 18, 14,  6,  4,  8, 20,  9, 13, 18, 10,  8,  7, 11,  4, 15,
       11,  3, 14, 20, 11,  1,  0, 15, 14, 13,  0, 18, 10, 23,  7,  3, 12,
       20,  3, 22,  1,  4, 12,  7, 13,  3,  1,  3,  0,  8,  9,  0, 23, 13,
       12,  9,  9, 20,  2, 11, 14, 15, 16, 23,  9,  3,  9, 17,  1,  1,  9,
       19, 20,  0,  4, 11

In [45]:
# Display the predictions and the ground truth visually.
def display_prediction (images, true_labels, predicted_labels):
    fig = plt.figure(figsize=(20, amount_of_samples +10))
    for i in range(len(true_labels)):
        truth = true_labels[i]
        prediction = predicted_labels[i]
        plt.subplot(amount_of_samples, 4,1+i)
        plt.axis('off')
        color='green' if truth == prediction else 'red'
        plt.text(30, 10, "Truth:        {0}\nPrediction: {1}".format(truth, prediction),
                 fontsize=12, color=color)
        plt.imshow(images[i])


In [ ]:
display_prediction(sample_images, ground_truth, predicted_categories)